
# **Optimización de Portafolio: Máximo Sharpe Ratio**

Este tutorial utiliza la optimización `MeanRisk` para encontrar la cartera con el **Máximo Sharpe Ratio** con ayuda de la libreria `skfolio` en Python, esta libreria ayuda la optimización de carteras construida sobre `scikit-learn`. Ofrece una interfaz unificada y herramientas compatibles con `scikit-learn` para construir, ajustar y validar modelos de cartera.

El objetivo de este análisis es:
- Cargar datos históricos de precios de acciones del **S&P 500**.
- Convertir precios en rendimientos.
- Aplicar técnicas de optimización para encontrar la cartera con el mejor **Sharpe Ratio**.

Antes de continuar, asegurémonos de que `skfolio` está instalado en el entorno de Python. Si no lo tienes instalado, ejecuta el siguiente comando:

In [1]:
!pip install skfolio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.5/734.5 kB 5.2 MB/s eta 0:00:00


## **Carga y preparación de datos**

En esta sección, importamos las bibliotecas necesarias y cargamos un conjunto de datos compuesto por los precios diarios de 20 activos del índice S&P 500. Los datos abarcan el período del 2 de enero de 1990 al 28 de diciembre de 2022.

Para garantizar la validez de la estrategia y evitar la **fuga de datos**, se divide el conjunto de datos en subconjuntos de entrenamiento y prueba sin realizar mezclas aleatorias.

In [2]:
# Importación de bibliotecas esenciales para la manipulación de datos y visualización
import numpy as np
from plotly.io import show
from sklearn.model_selection import train_test_split

# Importación de la biblioteca skfolio para análisis y optimización de carteras
from skfolio import Population, RiskMeasure
from skfolio.datasets import load_sp500_dataset  # Carga de datos históricos del S&P 500
from skfolio.optimization import InverseVolatility, MeanRisk, ObjectiveFunction
from skfolio.preprocessing import prices_to_returns  # Función para convertir precios en rendimientos

# Carga del conjunto de datos: precios históricos de 20 activos del S&P 500
prices = load_sp500_dataset()

# Transformación de precios a rendimientos lineales (también se podrían usar logarítmicos en algunos casos)
X = prices_to_returns(prices)

# División del conjunto de datos en entrenamiento (67%) y prueba (33%) sin mezclar los datos (shuffle=False)
X_train, X_test = train_test_split(X, test_size=0.33, shuffle=False)

# Vista preliminar de los primeros datos del conjunto de entrenamiento
print(X_train.head())

                AAPL       AMD       BAC       BBY       CVX        GE  \
Date                                                                     
1990-01-03  0.007576 -0.030303  0.008045  0.118056 -0.016229 -0.001876   
1990-01-04  0.003759 -0.015500 -0.021355 -0.012422 -0.012831 -0.005639   
1990-01-05  0.003745 -0.031996 -0.021821  0.000000 -0.014855 -0.009452   
1990-01-08  0.003731  0.000000  0.005633 -0.075472  0.009424  0.005725   
1990-01-09 -0.007435  0.016527  0.000000  0.000000 -0.007469 -0.020803   

                  HD       JNJ       JPM        KO       LLY       MRK  \
Date                                                                     
1990-01-03  0.003581  0.004072  0.033589 -0.014318  0.000000  0.015896   
1990-01-04  0.006244  0.002028  0.003991 -0.004993 -0.005557 -0.015647   
1990-01-05 -0.013298 -0.010408  0.003975 -0.008212 -0.010874 -0.020641   
1990-01-08 -0.009883  0.016944  0.000000  0.021159  0.000000  0.012839   
1990-01-09 -0.026316 -0.031026 -0.031

## **Modelo de Optimización de la Cartera**
En esta sección, se crea un modelo basado en la optimización de **Sharpe Ratio** utilizando la metodología **Media-Riesgo** (`Mean-Risk`).

Sharpe Ratio mide el **rendimiento ajustado al riesgo** de una cartera, comparando el **exceso de retorno** sobre la **desviación estándar** de los rendimientos. Un mayor Sharpe Ratio indica una mejor compensación entre riesgo y retorno.

Se utiliza la función objetivo de maximización del ratio con la desviación estándar como medida de riesgo. Además, se define un conjunto de parámetros (`portfolio_params`) que serán utilizados al generar la cartera óptima.

In [3]:
# Creación del modelo basado en la metodología Media-Riesgo (Mean-Risk)
model = MeanRisk(
    risk_measure=RiskMeasure.STANDARD_DEVIATION,  # Se define la desviación estándar como métrica de riesgo
    objective_function=ObjectiveFunction.MAXIMIZE_RATIO,  # Se establece la función objetivo para maximizar el Ratio de Sharpe
    portfolio_params=dict(name="Max Sharpe"),  # Se asigna un nombre a la cartera óptima
)

# Ajuste del modelo utilizando los datos de entrenamiento (rendimientos históricos)
model.fit(X_train)

# Obtención de los pesos óptimos asignados a cada activo en la cartera de Máximo Sharpe Ratio
model.weights_

array([9.43837536e-02, 1.23703225e-07, 4.32481914e-08, 1.20892854e-01,
       3.18418329e-02, 7.69682656e-08, 1.78420642e-04, 1.24117994e-01,
       8.50336289e-08, 2.77970034e-02, 1.31617983e-07, 1.49536745e-07,
       1.16362392e-01, 5.73881398e-02, 9.91607313e-07, 1.09506312e-01,
       8.64772579e-02, 1.84018669e-01, 1.34639296e-02, 3.35698407e-02])

## **Modelo de Referencia: Inversión Proporcional a la Volatilidad**
Para evaluar el desempeño del modelo de Máximo Sharpe Ratio, utilizamos una cartera de **volatilidad inversa** como referencia (`benchmark`).

Este enfoque asigna un mayor peso a los activos con menor volatilidad y un menor peso a los activos más volátiles. La lógica detrás de esta estrategia es que activos con menor variabilidad de precios pueden ofrecer un mejor ajuste riesgo-retorno en ciertos escenarios de mercado.


In [4]:
# Creación de un modelo de referencia basado en la estrategia de volatilidad inversa
benchmark = InverseVolatility(
    portfolio_params=dict(name="Inverse Vol")  # Se asigna un nombre a la cartera de referencia
)

# Ajuste del modelo utilizando los datos de entrenamiento
benchmark.fit(X_train)

# Obtención de los pesos óptimos asignados a cada activo según la estrategia de volatilidad inversa
benchmark.weights_

array([0.03306735, 0.02548697, 0.03551377, 0.0296872 , 0.06358463,
       0.05434705, 0.04742354, 0.07049715, 0.03882539, 0.06697905,
       0.05570808, 0.05576851, 0.04723274, 0.06351213, 0.05581397,
       0.0676481 , 0.02564642, 0.03970752, 0.05744543, 0.06610498])

## **Predicción y Evaluación de la Cartera**
En esta sección, utilizamos los modelos previamente entrenados para generar predicciones en el **conjunto de prueba**.

El método `.predict()` aplica la estrategia de optimización sobre los nuevos datos y devuelve un objeto de la clase `Portfolio`, el cual proporciona diversas métricas y herramientas de análisis para evaluar el desempeño de la cartera.

Se comparan los resultados del modelo de **Máximo Sharpe Ratio** y la referencia de **Volatilidad Inversa**, calculando métricas clave como el **Ratio de Sharpe Anualizado** y generando visualizaciones del desempeño de ambas estrategias.

In [5]:
# Predicción del modelo de Máximo Sharpe Ratio en el conjunto de prueba
pred_model = model.predict(X_test)

# Predicción del modelo de referencia (Volatilidad Inversa) en el conjunto de prueba
pred_bench = benchmark.predict(X_test)

# Conversión del objeto `Portfolio` a un array de retornos utilizando NumPy
returns_model = np.asarray(pred_model)  # Equivalente a pred_model.returns

# Cálculo del Ratio de Sharpe Anualizado para ambas estrategias
print("Sharpe Ratio (Max Sharpe):", pred_model.annualized_sharpe_ratio)
print("Sharpe Ratio (Inverse Volatility):", pred_bench.annualized_sharpe_ratio)

Sharpe Ratio (Max Sharpe): 1.039972499946779
Sharpe Ratio (Inverse Volatility): 1.0036976120249752


## **Análisis Comparativo de las Estrategias**
En esta sección, consolidamos las carteras pronosticadas en una estructura común para facilitar el análisis comparativo.

Para ello, utilizamos la clase `Population` de skfolio, que permite agrupar múltiples carteras y proporciona herramientas avanzadas para evaluar su desempeño.

Se analizan las diferencias en **composición de activos**, **rendimientos acumulados** y **métricas clave**, proporcionando una visión completa de ambas estrategias.



In [6]:
# Agrupación de las carteras pronosticadas (Máximo Sharpe y Volatilidad Inversa) en una única población
population = Population([pred_model, pred_bench])

# Visualización de la composición de cada cartera dentro de la población
population.plot_composition()  # Muestra la asignación de pesos en cada estrategia

Vamos a trazar los rendimientos acumulados de cada cartera:


In [7]:
# Comparación de los rendimientos acumulados a lo largo del tiempo
fig = population.plot_cumulative_returns()  # Genera la gráfica de comparación

# Mostrar la figura (opcional, necesario si se ejecuta en ciertos entornos de documentación)
show(fig)

Por último, mostremos el resumen completo de ambas estrategias evaluadas en el conjunto de pruebas:



In [8]:
# Mostrar un resumen completo con métricas de rendimiento y riesgo de ambas estrategias en el conjunto de prueba
population.summary()

,Max Sharpe,Inverse Vol
Mean,0.073%,0.064%
Annualized Mean,18.43%,16.06%
Variance,0.012%,0.010%
Annualized Variance,3.14%,2.56%
Semi-Variance,0.0063%,0.0053%
Annualized Semi-Variance,1.58%,1.33%
Standard Deviation,1.12%,1.01%
Annualized Standard Deviation,17.72%,16.00%
Semi-Deviation,0.79%,0.73%
Annualized Semi-Deviation,12.58%,11.54%


## **Conclusión**
El análisis realizado sobre el **conjunto de prueba** demuestra que la cartera optimizada para **Máximo Sharpe Ratio** supera a la estrategia de **Volatilidad Inversa** en términos de **retorno medio** y métricas de **eficiencia ajustada al riesgo**, incluyendo Sharpe Ratio.

Sin embargo, en términos de **volatilidad absoluta** y medidas de **pérdida potencial**, la estrategia de Volatilidad Inversa presenta un desempeño más conservador, lo que podría ser preferible en escenarios de alta incertidumbre.

Este ejemplo ilustra cómo las diferentes metodologías de optimización pueden impactar la asignación de activos y el perfil de riesgo-retorno de una cartera.

Para explorar conceptos más avanzados y estrategias adicionales, consulte la **guía del usuario** y los **ejemplos prácticos** disponibles en la documentación oficial de `skfolio`:
🔗 https://skfolio.org/


